In [1]:
import pandas as pd
from functools import reduce

In [6]:
df_sda = pd.read_excel('./db/Sdas_HTS_AR.xlsx')
df_sda = df_sda.convert_dtypes()
df_sda.dropna(how='all', inplace=True)
df_sda = df_sda.astype('string')
df_sda.fillna('-', inplace=True)
df_sda = df_sda.loc[:, ['Delivery', 'Material', 'QTY', 'Invoice_Number', 'HTS_GTS', 'HTS_WWClass', 'SDA', 'Mode', 'PL', 'BU']]
df_sda

,Delivery,Material,QTY,Invoice_Number,HTS_GTS,HTS_WWClass,SDA,Mode,PL,BU
0,8023484935,593W0EC#AC8,1,-,-,84713012190L,Missing Invoice on GTS Report,Ocean,AN,Computing
1,8023188064,UD4W3AAE,1,-,-,SERVICEONLY,Missing Invoice on GTS Report,Ocean,M2,Computing
2,8022148818,507Q3EP#AC8,1,9323751142,84713019900D,84713012991G,HTS-GTS <> HTS-WWClass,Air,IK,Computing


In [5]:

df_group_sda = df_sda.groupby(['SDA', 'Mode', 'BU'])['Delivery'].count().reset_index()
df_group_sda

,SDA,Mode,BU,Delivery
0,HTS-GTS <> HTS-WWClass,Air,Computing,1
1,Missing Invoice on GTS Report,-,Computing,2


In [4]:
df_delay_reason = pd.read_excel('./db/Delay_reasons.xlsx')
df_delay_reason.head()

,SDA,Delay reason,Delay Sub-reason
0,HTS-GTS <> HTS-WWClass,Classification Disputes,Disputed by Service Provider
1,Missing Invoice on GTS Report,Invoice Missing,GTS web portal
2,Blank HTS on GTS Report,Classification Missing,Missing on Invoice
3,Missing HTS on GTS Report,Classification Missing,Missing on Invoice


In [5]:
df_sda = df_sda.merge(df_delay_reason, on='SDA', how='inner')
df_sda['key'] = df_sda['SDA'] + df_sda['Mode'] + df_sda['BU']
list_sda = list(df_sda['key'].unique())
list_sda

[nan,
 'Missing Invoice on GTS Report-Computing',
 'HTS-GTS <> HTS-WWClassAirComputing']

In [6]:
dict_sda = {}
invoices = []
list_request_sda = []
for k in list_sda:
    df_sda_filter = df_sda[df_sda['key'] == k]
    dict_sda['Delivery']  = list(df_sda_filter['Delivery'])
    #dict_sda['Material'] = list(df_sda_filter['Material'])
    dict_sda['QTY'] = list(df_sda_filter['QTY'])
    #dict_sda['Invoice_Number'] = list(df_sda_filter['Invoice_Number'])
    for i in list(df_sda_filter['Invoice_Number'].unique()):
        invoice = {}
        invoice['Invoice_Number'] = i
        df_sda_filter_invoice = df_sda_filter[df_sda_filter['Invoice_Number'] == i]
        invoice['Material'] = list(df_sda_filter_invoice['Material'])
        invoice['QTY'] = list(df_sda_filter_invoice['QTY'])
        invoice['PL'] = list(df_sda_filter_invoice['PL'])
        invoices.append(invoice)
    dict_sda['HTS_GTS'] = list(df_sda_filter['HTS_GTS'])
    dict_sda['HTS_WWClass'] = list(df_sda_filter['HTS_WWClass'])
    dict_sda['SDA'] = list(df_sda_filter['SDA'].drop_duplicates())
    dict_sda['Mode'] = list(df_sda_filter['Mode'].drop_duplicates())
    #dict_sda['PL'] = list(df_sda_filter['PL'])
    dict_sda['BU'] = list(df_sda_filter['BU'].drop_duplicates())
    dict_sda['Delay reason'] = list(df_sda_filter['Delay reason'].drop_duplicates())
    dict_sda['Delay Sub-reason'] = list(df_sda_filter['Delay Sub-reason'].drop_duplicates())
    list_request_sda.append(dict_sda.copy())
    
print(list_request_sda)
print('-' * 100)
print(invoices)


[{'Delivery': [], 'QTY': [], 'HTS_GTS': [], 'HTS_WWClass': [], 'SDA': [], 'Mode': [], 'BU': [], 'Delay reason': [], 'Delay Sub-reason': []}, {'Delivery': [8023188064], 'QTY': [1], 'HTS_GTS': [nan], 'HTS_WWClass': ['SERVICEONLY'], 'SDA': ['Missing Invoice on GTS Report'], 'Mode': ['-'], 'BU': ['Computing'], 'Delay reason': ['Invoice Missing'], 'Delay Sub-reason': ['GTS web portal']}, {'Delivery': [8022148818], 'QTY': [1], 'HTS_GTS': ['84713019900D'], 'HTS_WWClass': ['84713012991G'], 'SDA': ['HTS-GTS <> HTS-WWClass'], 'Mode': ['Air'], 'BU': ['Computing'], 'Delay reason': ['Classification Disputes'], 'Delay Sub-reason': ['Disputed by Service Provider']}]
----------------------------------------------------------------------------------------------------
[{'Invoice_Number': nan, 'Material': [], 'QTY': [], 'PL': []}, {'Invoice_Number': 9323751142.0, 'Material': ['507Q3EP#AC8'], 'QTY': [1], 'PL': ['IK']}]


In [24]:
list_request_sda[0]['QTY']

[24, 72]

In [27]:
for k in range(0, len(list_request_sda)):
    print(k)

0
1
2


In [31]:

for k in range(0, len(list_request_sda)):
    qty = reduce(lambda a, b: a + b, list_request_sda[k]['QTY'])
    print(list_request_sda[k]['BU'][0], list_request_sda[k]['Mode'][0], qty, list_request_sda[k]['Delivery'])

Printing Ocean 96 [8020981243, 8021074551]
Computing Ocean 810 [8021075477]
Computing Air 1 [8022148818]


In [33]:
l1 = [1, 2, 4, '3']
l2 = ['<', 'x', 43, 'hola']
l3 = [12, 22, 42, 'chao']

print('HTS_GTS', '\t', 'HTS_WWClass', '\t', 'SDA', '\n')
for i, j, k in zip(l1, l2, l3):
    print(i, '\t', j, '\t', k, '\n')

HTS_GTS 	 HTS_WWClass 	 SDA 

1 	 < 	 12 

HTS_GTS 	 HTS_WWClass 	 SDA 

2 	 x 	 22 

HTS_GTS 	 HTS_WWClass 	 SDA 

4 	 43 	 42 

HTS_GTS 	 HTS_WWClass 	 SDA 

3 	 hola 	 chao 

